# OpenRefine Client

Notebook demonstrating how to control OpenRefine via a Python client.

Use the `dbutlerdb/refine-client-py` fork of `PaulMakepeace/refine-client-py` for Python3 support.

## Getting Started

To start, ensure that the OpenRefine application server is running. You can start it from the notebook homepage (`New -> OpenRefine Session`.

The server connection is looked for on the default port 3333. This can be hardcoded as part of the `nbopenrefineder` OpneRefine start-up command.

In [ ]:
from open.refine import refine

server = refine.RefineServer()
orefine = refine.Refine(server)

We can list any projects that currently exist:

In [13]:
orefine.list_projects().items()

dict_items([('2449490768888', {'name': 'Untitled', 'created': '2019-01-02T17:12:51Z', 'modified': '2019-01-02T17:14:55Z', 'creator': '', 'contributors': '', 'subject': '', 'description': '', 'rowCount': 0, 'customMetadata': {}}), ('2209465772212', {'name': 'Untitled', 'created': '2019-01-02T17:12:19Z', 'modified': '2019-01-02T17:14:55Z', 'creator': '', 'contributors': '', 'subject': '', 'description': '', 'rowCount': 0, 'customMetadata': {}})])

## Creating Projects

Projects can be created via the API from a source data file. 

In [8]:
%%capture
try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd

In [38]:
#Create a simple test data file as a CSV file

#via https://github.com/dbutlerdb/refine-client-py/blob/master/tests/data/duplicates.csv
data = '''email,name,state,gender,purchase
danny.baron@example1.com,Danny Baron,CA,M,TV
melanie.white@example2.edu,Melanie White,NC,F,iPhone
danny.baron@example1.com,D. Baron,CA,M,Winter jacket
ben.tyler@example3.org,Ben Tyler,NV,M,Flashlight
arthur.duff@example4.com,Arthur Duff,OR,M,Dining table
danny.baron@example1.com,Daniel Baron,CA,M,Bike
jean.griffith@example5.org,Jean Griffith,WA,F,Power drill
melanie.white@example2.edu,Melanie White,NC,F,iPad
ben.morisson@example6.org,Ben Morisson,FL,M,Amplifier
arthur.duff@example4.com,Arthur Duff,OR,M,Night table'''


fn = 'test.csv'

with open(fn,'w') as f:
    f.write(data)

Either copy and paste the above data into a new OpenRefine project as a *clipboard* item or load it in as a CSV data file.

We should now be able to see the project via the project index list:

In [132]:
#For some reason, the project does not appear to get named?
orefine.list_projects()

{'2165169842672': {'name': 'clipboard',
  'created': '2019-01-02T17:38:54Z',
  'modified': '2019-01-02T18:04:55Z',
  'creator': '',
  'contributors': '',
  'subject': '',
  'description': '',
  'rowCount': 10,
  'customMetadata': {},
  'importOptionMetadata': [{'guessCellValueTypes': False,
    'ignoreLines': -1,
    'processQuotes': True,
    'fileSource': '(clipboard)',
    'encoding': '',
    'separator': ',',
    'storeBlankCellsAsNulls': True,
    'storeBlankRows': True,
    'skipDataLines': 0,
    'includeFileSources': False,
    'headerLines': 1,
    'limit': -1,
    'projectName': 'clipboard'}]}}

## Opening an OpenRefine Project

To open an OpenRefine project, we need to get it's key value from the project list. (Note that different projects may have the same name.)

In [44]:
dir(orefine)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'default_options',
 'get_project_id',
 'get_project_name',
 'list_projects',
 'new_project',
 'open_project',
 'server',
 'set_options',
 'set_project_name']

In [133]:
KEY = '2165169842672'  #Need to use a valid key
KEY = list(orefine.list_projects().keys())[0]

p = orefine.open_project(KEY)

In [134]:
#Inspect the column names
p.columns

['Email', 'name', 'NAME', 'NAME2', 'state', 'gender', 'purchase']

In [135]:
#Rename a column

col = p.columns[-1]
p.rename_column(col,'{}2'.format(col))
p.columns, p.column_order

(['Email', 'name', 'NAME', 'NAME2', 'state', 'gender', 'purchase2'],
 {'Email': 0,
  'name': 1,
  'NAME': 2,
  'NAME2': 3,
  'state': 4,
  'gender': 5,
  'purchase': 6,
  'purchase2': 6})

*Hmmm, it looks like there's a duplicate there?*

### Adding A Column

OpenRefine allows you to add a column derived from one or more other columns.

```
.add_column(oldColumn, newColumn, expression="value", column_insert_index=None, on_error='set-to-blank'])
```

The `value` keyword denotes using the cell values from the original column.

In [68]:
p.add_column('name', 'name2', column_insert_index=2)

{'code': 'ok',
 'historyEntry': {'id': 1546452832068,
  'description': 'Create new column NAME2 based on column name by filling 10 rows with value',
  'time': '2019-01-02T18:04:03Z'}}

In [69]:
#Add a column based on the name
p.add_column('name', 'NAME', "toUppercase(value)")

{'code': 'ok',
 'historyEntry': {'id': 1546452926788,
  'description': 'Create new column NAME based on column name by filling 10 rows with toUppercase(value)',
  'time': '2019-01-02T18:04:17Z'}}

For a list of supported GREL (Google Refine Expression Language) commands, see [here](https://github.com/OpenRefine/OpenRefine/wiki/GREL-String-Functions).

### Delete A Column

The `remove column` OpenRefine option doesn't appear to be supported?

## Export CSV

In [81]:
import pandas as pd
from io import StringIO

pd.read_csv( StringIO( p.export(export_format='csv') ) )

,email,name,state,gender,purchase
0,danny.baron@example1.com,Danny Baron,CA,M,TV
1,melanie.white@example2.edu,Melanie White,NC,F,iPhone
2,danny.baron@example1.com,D. Baron,CA,M,Winter jacket
3,ben.tyler@example3.org,Ben Tyler,NV,M,Flashlight
4,arthur.duff@example4.com,Arthur Duff,OR,M,Dining table
5,danny.baron@example1.com,Daniel Baron,CA,M,Bike
6,jean.griffith@example5.org,Jean Griffith,WA,F,Power drill
7,melanie.white@example2.edu,Melanie White,NC,F,iPad
8,ben.morisson@example6.org,Ben Morisson,FL,M,Amplifier
9,arthur.duff@example4.com,Arthur Duff,OR,M,Night table


## Creating an OpenRefine Project

*This doesn't seem to work?*

*It would be useful to have a simple recipe for creating a project from a pandas dataframe.*

In [118]:
import os

#Create an OpenRefine project from the data file
#USe the absolute path to the file
p=orefine.new_project(project_file=os.path.abspath(fn), project_name='Test 1', project_file_name=fn)

#Do we have to mediate this via a file transfer? eg could we go more directly from a pandas dataframe somehow?

In [119]:
#For some reason, the project does not appear to get named?
#There also can be a delay before the index listing shows that the data has been loaded?
orefine.list_projects()

{'1860845873071': {'name': 'Untitled',
  'created': '2019-01-02T18:27:13Z',
  'modified': '2019-01-02T18:27:27Z',
  'creator': '',
  'contributors': '',
  'subject': '',
  'description': '',
  'rowCount': 10,
  'customMetadata': {},
  'importOptionMetadata': [{'storeBlankRows': True,
    'includeFileSources': False,
    'skipDataLines': 0,
    'guessCellValueTypes': False,
    'headerLines': 1,
    'ignoreLines': -1,
    'processQuotes': True,
    'fileSource': 'test.csv',
    'separator': ',',
    'storeBlankCellsAsNulls': True}]},
 '2165169842672': {'name': 'clipboard',
  'created': '2019-01-02T17:38:54Z',
  'modified': '2019-01-02T18:04:55Z',
  'creator': '',
  'contributors': '',
  'subject': '',
  'description': '',
  'rowCount': 10,
  'customMetadata': {},
  'importOptionMetadata': [{'guessCellValueTypes': False,
    'ignoreLines': -1,
    'processQuotes': True,
    'fileSource': '(clipboard)',
    'encoding': '',
    'separator': ',',
    'storeBlankCellsAsNulls': True,
    'sto

### Delete a Project

Take care with this one — there is no warning...

In [120]:
p.delete()

True